The first program to look at is the code sd_movement. With this program it is possible to scan and visualize the field in two dimensions.
It relys on other programs which have to be included first.

In [5]:
include("examplesalex/stages.jl")
include("examplesalex/measurement.jl")
include("examplesalex/plot.jl")


correctPosition (generic function with 1 method)

In the following cell, the connection between the four motors and the computer is established. Each motor is given a name and sorted. The motors that are not used for this measurement are disconnected again and can be controlled by another person.

In [ ]:
### Connect to the Motor "Bigger Chungus" ###
devcount, devenum, enumnames = setupDevices(ENUMERATE_PROBE | ENUMERATE_NETWORK,b"addr=134.61.12.184")

D = openDevices(enumnames,stagenames)
checkOrdering(D,stagenames)
closeDevices(D[1:2])


The next step is to select the basic settings of the VNA and connect it to the computer. 
InstrumentSimplifiedSetup sets the instrument according to the selected settings.
A name for the measurement is also selected.
The res parameter specifies the resolution of the scan by determining the distance the motors travel vertically after each horizontal scan. The number must be between 1 and 7, the higher the number the lower the resolution. More information about this will be shown later when explaining the measuremnt.jl code.

In [ ]:


#Basic VNA settings
power=-20
f_center::Float64 = 20e9
f_span::Float64 = 3e9
sweepPoints::Integer = 128
ifbandwidth::Integer = 100e3


#Resolution of the scan, a number between 1 and 7 needs to be selected. The bigger the number the lower the resolution 
res = 7
name::String = "2D_CristalRB3-07.jld2"

#Connect to the VNA and start basics setup
vna = connectVNA()
vnaParam = instrumentSimplifiedSetup(vna; calName=cals[:c3GHz_NEW], power=power, center=f_center, span=f_span, sweepPoints=sweepPoints, ifbandwidth=ifbandwidth)



Now everything is ready to start a scan. The twoDMeasurement function tells the motors how to move, decides when to make the measurements, saves the data to variables and counts the time the whole process takes. 
The data is stored in a JLD2 file for later access, and the motors are disconnected because they are not needed to analyze the data.

In [ ]:

@time S, f, pos_BIGGER, pos_BIG, posSet = twoDMeasurement(vna, 0, 18000; speed=2000, speedSetup=2000, res=res, sweepPoints=sweepPoints)

meas = Measurement2D("", vnaParam, f, S, pos_BIGGER, pos_BIG, posSet)
saveMeasurement(meas; name=name)

closeDevices(D)

Now the stored data can be retrieved via the readMeasurment and visualized with the plot... functions.
The transform function has been added to change the way the data is stored. As will be shown later in the measurement.jl code, the S11 parameters are stored as a matrix where each entry is a vector whose length is equal to the number of SweepPoints. This format was chosen because the x and y dimensions are stored as columns and rows of a matrix. Each measurement point in space has an x and y coordinate, as well as the number of SweepPoints in the third dimension. 
To visualize the scanned field, the format of the S11 was changed to a vector where each entry is an x-y matrix. In this way, one can clearly see the different images for different frequencies.

In [ ]:

data = readMeasurement("C:/Users/deslis.INST3/JuXIMC.jl/2D_CristalRB3-07_500_2023-07-18_1.jld2_2023-07-26_4.jld2")
transData = transform(data, sweepPoints, res)

plotPoints(data, sweepPoints, res, transData)
plotGaussianFit2D(data, sweepPoints, transData)



Now we iterate through the individual frequencies and visualize the data as heatmaps.
To see the result clearly, all plots are saved as gif.

In [ ]:

anim = @animate for freqIndex in 1:sweepPoints	
    plotHeatmap2D(data, freqIndex)
    
end
gif(anim, "anim_"*name*".gif", fps = 10)


A helpfull tip if a scan has to be interupted or is otherwise unfinished is the following cell. 
The unfinished program is not able to deleted the saved traces on the memory of the VNA, which causes VNA erros if a new scan is started.
In order to avoid this the traces have to be cleared manually using the next lines. The number 45 is picked by random and can be changed but it hast to be high enough to deleted all saves traces.


In [ ]:

for i in 1:45
    deleteTrace(vna, i)
end

It takes several arguments, most of which should be self-explanatory.
The first settings bind the names of the motors to their numbers to show more clearly which motor is moving.
The setStepSize function uses the selected resolution to calculate the step size and the number of vertical steps (vNum).  
Resolution contains 7 numbers between 1 and 20, which are selected because vNum should be an integer and the area to be scanned (by me) is set to 20 cm.

In [ ]:
function twoDMeasurement(socket::TCPSocket, startPos::Integer, endPos::Integer; speed::Integer=1000, speedSetup::Integer=1000, res::Integer, sweepPoints::Integer)
    

    BigChungus = 3
    BiggerChungus = 4
       
    stepSize, vNum = setStepSize(res)
    
    if res > 7
        error("Resolution is too low, choose a number between 1 - 7 ")
    end

    println("A resolution of "*string(res)*" was chosen, this corresponds to "*string(resolution[res])*" mm per vertical movement and "*string(Int(vNum))*" horizontal lines in the resultuing plot")
    

In the next lines some variables are defined as well as some important data like the frequencies and the given positions where a measurement should be performed are stored.

In [ ]:
    pos_data_BIGGER = Vector{Position}(undef, 0)
    pos_data_BIG = Vector{Position}(undef, 0)
    f_data = getFreqAsBinBlockTransfer(vna)

    posSet = getMeasurementPositions(startPos, endPos; stepSize=stepSize)
    posSetLen = length(posSet)::Int64
    S_data = Matrix{Vector{ComplexF64}}(undef,  vNum, length(posSet))

Before a measurement can take place, both motors must move to their starting positions, which is done by the following cell. It is also possible to change this function so that it only performs a 1D scan, in which case the following lines would have to be changed. 
However, I do not recommend this, because there are other programs that are made for 1D scans.

In [ ]:

    #      for 2D
    for i in BigChungus:BiggerChungus
        
        setSpeed(D[i], speedSetup)  
        commandMove(D[i], startPos, 0) 
        commandWaitForStop(D[i])       
        
    end
    

Now the motors are moved depending on the position.
The scan start is in the upper right corner, then it scans once to the upper left, then the system moves down where it scans from left to right. 
This results in three scenarios. 

First: The bead is in the upper right corner and must perform an initial scan before moving down.
Second: The bead is on the left side of the area to be scanned, so it must first be moved down and then from left to right. 
Third: The bead is on the right side, but must first be moved down before it is moved to the left.

In [ ]:

    #movement of the motors
    for y in 1:vNum
        println("round ", y)
        currentPos_BIG = getPos(D[BigChungus])
        push!(pos_data_BIG, currentPos_BIG)
        
        current = 1
        if y == 1    
            commandMove(D[BiggerChungus], endPos, 0)
            
        elseif y % 2 == 0
            commandMove(D[BigChungus], stepSize*y, 0)
            commandWaitForStop(D[BigChungus])
            
            commandMove(D[BiggerChungus],  startPos, 0)
            
        else
            commandMove(D[BigChungus], stepSize*y, 0)
            commandWaitForStop(D[BigChungus])
            
            commandMove(D[BiggerChungus], endPos, 0)
            
        end
    

While the motor is moving, the program checks whether the previously defined measuring point is passed. If this is the case, a measurement is made via the storeTraceInMemory function. 
This function takes the data corresponding to the visible trace and stores it in the VNA's memory. This is repeated for all scans. 
The direction of the scan causes some complications, as we will see later.

In [ ]:
        while true
            currentPos_BIGGER = getPos(D[BiggerChungus])
            
            # Check wether the current position has passed the intended point of measurement.
            # The condition if a point has been passed is dependent on the direction of travel.
        
            if y % 2 == 0
                passed = isSmallerEqPosition(currentPos_BIGGER, Position(posSet[length(posSet)+1-current], 0))
            else
                passed = isGreaterEqPosition(currentPos_BIGGER, Position(posSet[current], 0))
            end
            # If a point has been passed, perform a measurement
            if passed
                storeTraceInMemory(socket, current)
                push!(pos_data_BIGGER, currentPos_BIGGER)
                current += 1
                if current == length(posSet) +1  break end
            end
        end

The next cell takes the stored data from memory and stores it in S_data.

In [ ]:
        
        # Read the data from Memory
        for x in 1:length(posSet)
            
            if y % 2 == 0
                S_params = getTraceFromMemory(socket, x)
                
                S_data[y, length(posSet) - x + 1] = S_params
                
            else
                S_params = getTraceFromMemory(socket, x)
                S_data[y,x] = S_params
            end
        end
    end

    return (S_data, f_data, pos_data_BIGGER, pos_data_BIG, posSet)
end
       

One of the biggest problems we have is recording the data. As mentioned earlier, it depends on the direction of the scan. 
The moment the bead moves past one of the measurement points, both the communication of the VNA with the computer and the data recording itself takes some time. During this time, the bead continues to move, causing the point to shift. This shift is exacerbated by the fact that the scan is made in one direction in one pass and in the other direction in the other pass. 
This effect is shown in the resulting plot.

In [ ]:
freqIndex = argmin(abs.(data.freq .- 20*10^9))
plotHeatmap2D(data, freqIndex)


The next program can be used to perform 1D measurements. This is often useful for tests and simple field scans.
Since only one motor is moved, it is helpful to assign it directly to a variable, in our case the letter D.

In [ ]:
include("../examplesdom/stages.jl")
include("measurement.jl")
include("plot.jl")


### Connect to the Motor "Bigger Chungus" ###
devcount, devenum, enumnames = setupDevices(ENUMERATE_PROBE | ENUMERATE_NETWORK,b"addr=134.61.12.184")

D = openDevices(enumnames,stagenames)
checkOrdering(D,stagenames)
D = D[4]


### Connect to the VNA ###
power=-20
f_center::Float64 = 20e9
f_span::Float64 = 3e9
sweepPoints::Integer = 128
ifbandwidth::Integer = 100e3
measurement::String = "CH1_S11_1"

vna = connectVNA()
vnaParam = instrumentSimplifiedSetup(vna; calName=cals[:c3GHz_NEW], power=power, center=f_center, span=f_span, sweepPoints=sweepPoints, ifbandwidth=ifbandwidth)

@time S, f, pos, posSet = getContinousMeasurement(vna, 0, 20000; speed=2000, speedSetup=2000, stepSize=500)
meas = Measurement("speedtest", vnaParam, f, S, pos, posSet)
saveMeasurement(meas; filename="metalwire_3GHz.data")

plotHeatmap(meas)
plotGaussianFit(meas)



A simple signal to noise ration analysis is possible with the next program. 

In [ ]:
include("stages.jl")
include("measurement.jl")
include("plot.jl")


### Connect to the Motor "Bigger Chungus" ###
devcount, devenum, enumnames = setupDevices(ENUMERATE_PROBE | ENUMERATE_NETWORK,b"addr=134.61.12.184")

D_all = openDevices(enumnames,stagenames)
checkOrdering(D_all,stagenames)
#closeDevices(D[1:2])
D = D_all[4]


powerStep = [-20, -15, -10, -5, 0, 5, 10, 14]    
bead = "CristalRB3-07"
date = "2023-07-12"

plot()

for i in eachindex(powerStep)

    global vna = connectVNA()

    power= powerStep[i]
    println(power)
    f_center::Float64 = 20e9
    f_span::Float64 = 3e9
    sweepPoints::Integer = 128
    ifbandwidth::Integer = 100e3

    
    name::String = bead*"_SNR_P"* string(powerStep[i])
    
    vnaParam = instrumentSimplifiedSetup(vna; calName=cals[:c3GHz_NEW], power=power, center=f_center, span=f_span, sweepPoints=sweepPoints, ifbandwidth=ifbandwidth)

    @time S, f, pos, posSet = getContinousMeasurement(vna, 0, 18000; speed=2000, speedSetup=2000, stepSize=500)
    meas = Measurement("", vnaParam, f, S, pos, posSet)
    saveMeasurement(meas; name= "/inst3/data/Benutzer/deslis/Desktop/Hiwi-Job/"*name*".jld2")
      
    disconnectVNA(vna)
end

closeDevices(D_all)


plot()
for i in eachindex(powerStep)
    power=  powerStep[i]
    data = readMeasurement(bead*"_SNR_P"*string(power)*date*".jld2")
    plotGaussianFit(data, power)
end
png("powerComp4")

#Power to noise ratio

SNR_list = []
power_list = []

data = meas

plot()
for i in eachindex(powerStep)
    power = powerStep[i]
    data = readMeasurement("//inst3/data/Benutzer/deslis/Desktop/Hiwi-Job/Data/2D_CristalRB3-07_SNR_P"*string(power)*"_2023-06-06_1.jld2"

    E = calcFieldProportionality(data)*sqrt(10^(power/10)/1000) .*1000
    y = dropdims(sum(E, dims=2), dims=2)[begin+1:end]


    SNR = maximum(y) / minimum(y)
    push!(SNR_list, SNR)
    println(SNR_list)
end

scatter!(powerStep, SNR_list, xlabel = "Power Level [dBm]", ylabel = "Signal to noise ratio")
png("powerComp3_SNR")
